In [10]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd 
import sys
import pickle
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
from sklearn.metrics import pairwise_distances
from sklearn.kernel_ridge import KernelRidge
import math

workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)

import allensdk.core.json_utilities as ju
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

from mcmodels.core import VoxelModelCache
from mcmodels.core.utils import get_structure_id,get_ordered_summary_structures
from mcmodels.core.connectivity_data import get_connectivity_data
from mcmodels.models.crossvalidation import get_nwloocv_predictions_multimodel_merge_dists
from mcmodels.utils import nonzero_unique #, unionize
from mcmodels.models.crossvalidation import get_best_hyperparameters,get_loss_best_hyp,get_loss#get_loocv_predictions,get_loss#get_best_hyperparameters,get_loss_best_hyp,get_loocv_predictions,get_loss
from mcmodels.core.utils import get_leaves_ontologicalorder, get_indices, get_indices2,get_eval_indices,screen_index_matrices,screen_index_matrices2,screen_index_matrices3#get_cre_status,get_minorstructure_dictionary,get_leaves_ontologicalorder
from mcmodels.core.utils import get_indices_2ormore
from mcmodels.regressors import NadarayaWatson
from mcmodels.core.plotting import plot_loss_surface,plot_loss_scatter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
#read data
TOP_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
INPUT_JSON = os.path.join(TOP_DIR, 'input_011520.json')
EXPERIMENTS_EXCLUDE_JSON = os.path.join(TOP_DIR, 'experiments_exclude.json')
FILE_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
OUTPUT_DIR = os.path.join(FILE_DIR, 'output')

input_data = ju.read(INPUT_JSON)
manifest_file = input_data.get('manifest_file')
manifest_file = os.path.join(TOP_DIR, manifest_file)
experiments_exclude = ju.read(EXPERIMENTS_EXCLUDE_JSON)

#its unclear why the hyperparameters are loaded from the output directory
cache = VoxelModelCache(manifest_file=manifest_file)
major_structures = input_data.get('structures')
major_structure_ids = [get_structure_id(cache, s) for s in major_structures]
data_info = pd.read_excel('/Users/samsonkoelle/alleninstitute/Whole Brain Cre Image Series_curation only.xlsx', 'all datasets curated_070919pull')
data_info.set_index("id", inplace=True)
ontological_order = get_ordered_summary_structures(cache)

mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
st = mcc.get_structure_tree()
ai_map = st.get_id_acronym_map()
ia_map = {value: key for key, value in ai_map.items()}

#regionalize voxel model: compare with regional model
#regional parameters
cre = None
eid_set=None
high_res=False
threshold_injection = False

COARSE_STRUCTURE_SET_ID = 2#3#2#167587189#3#2(old)
DEFAULT_STRUCTURE_SET_IDS = tuple([COARSE_STRUCTURE_SET_ID])
tree = cache.get_structure_tree()
default_structures = tree.get_structures_by_set_id(DEFAULT_STRUCTURE_SET_IDS)
default_structure_ids = [st['id'] for st in default_structures if st['id'] != 934]
#cre= True

In [12]:
#info = cache.get_experiments(dataframe= [628236463])

In [18]:
COARSE_STRUCTURE_SET_ID = 167587189#2#167587189#3#2(old)
DEFAULT_STRUCTURE_SET_IDS = tuple([COARSE_STRUCTURE_SET_ID])
tree = cache.get_structure_tree()
default_structures = tree.get_structures_by_set_id(DEFAULT_STRUCTURE_SET_IDS)
print(len(default_structures))#, default_structures

316


In [11]:
#default_structures_new = default_structures.copy()

In [12]:
#ds_id_new = [default_structures_new[i]['id'] for i in range(len(default_structures_new))]

In [16]:
#len(np.where(np.isin(info['primary_injection_structure'],ds_id_new ))[0])

In [17]:
#.shape
#len(np.where(np.isin(info['primary_injection_structure'],ontological_order ))[0])

In [13]:
connectivity_data = get_connectivity_data(cache, major_structure_ids, experiments_exclude, remove_injection = False,structure_set_id = 167587189)

connectivity_data.get_injection_hemisphere_ids()
connectivity_data.align()
connectivity_data.get_centroids()
connectivity_data.get_data_matrices(default_structure_ids)
connectivity_data.get_crelines(data_info)
with open('data/info/leafs.pickle', 'rb') as handle:
    leafs = pickle.load(handle)
    
connectivity_data.ai_map = ai_map
connectivity_data.get_summarystructures(data_info)
connectivity_data.leafs = leafs

512
703
1089
1097
315
313
354
698
771
803
477
549


In [5]:
a = np.zeros(12)
for i in range(12):
    a[i] = np.unique(len(connectivity_data.structure_datas[major_structure_ids[i]].summary_structures))

In [6]:
a.sum()

1751.0

In [23]:
a = np.zeros(12)
for i in range(12):
    a[i] = np.unique(len(connectivity_data.structure_datas[major_structure_ids[i]].summary_structures))

In [25]:
a.sum()

1751.0

In [27]:
b = np.zeros(12)
for i in range(12):
    b[i] = np.unique(len(connectivity_data.leafs[major_structure_ids[i]]))

In [29]:
b.sum()

1751.0

In [7]:
len(ontological_order)

291

In [31]:
len(ontological_order)

315

In [33]:
len(ontological_order_leaves)

566

In [8]:
ontological_order_leaves = get_leaves_ontologicalorder(connectivity_data, ontological_order)
len(ontological_order_leaves)

541

In [29]:
len(ontological_order), len(np.unique(ontological_order))

(315, 315)

In [89]:
#major division segregation is legacy code but convenient for fast cross validation in major division model
#experiments_minor_structures = get_summarystructure_dictionary(connectivity_data, data_info)
#get leaves in ontological order.  Where leafs don't exist, uses summary structure
ontological_order_leaves = get_leaves_ontologicalorder(connectivity_data, ontological_order)
#Key isn't affected by which experiment we choose. This allows default masking to be inherited from the AllenSDK.
sid0 = list(connectivity_data.structure_datas.keys())[0]
eid0 = list(connectivity_data.structure_datas[sid0].experiment_datas.keys())[0]
#Identify keys denoting which voxels correspond to which structure in the ipsi and contra targets.
contra_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(structure_ids=ontological_order_leaves, hemisphere_id=1)
ipsi_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(structure_ids=ontological_order_leaves, hemisphere_id=2)

#contra_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(structure_ids=ontological_order, hemisphere_id=1)
#ipsi_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(structure_ids=ontological_order, hemisphere_id=2)
#get average intensities of projection structures given ipsi and contra keys
#source_key = ontological_order #only relevant here when injection needs to be unionized, but currently a required argument
ipsi_target_regions, ipsi_target_counts = nonzero_unique(ipsi_targetkey, return_counts=True)
contra_target_regions, contra_target_counts = nonzero_unique(contra_targetkey, return_counts=True)

target_order = lambda x: np.array(ontological_order_leaves)[np.isin(ontological_order_leaves, x)]
permutation = lambda x: np.argsort(np.argsort(target_order(x)))
targ_ids = np.concatenate([ipsi_target_regions[permutation(ipsi_target_regions)],
                           contra_target_regions[permutation(contra_target_regions)]])

IndexError: index 565 is out of bounds for axis 0 with size 564

In [92]:
#problem, ipsi_target_regions is too small
#redundancy in ipsi_targetkey
#redundancy in ontological_order_leaves
print(len(np.unique(ontological_order_leaves))) 
print(np.unique(ipsi_targetkey).shape)
print(ipsi_target_regions.shape)
print(ontological_order_leaves.shape)
print(np.unique(ontological_order_leaves).shape)

564
(565,)
(564,)
(566,)
(564,)


In [93]:
oll = get_leaves_ontologicalorder(connectivity_data, ontological_order)

In [94]:
#1098 is present twice
Counter(oll)[1098]

2

In [110]:
Counter(oll)[1107]

2

In [116]:
ia_map[1107], ia_map[1098], ia_map[395]

('MDRNv', 'MDRNd', 'MDRN')

In [115]:
ontological_order[277]

395

In [103]:
from mcmodels.core import Mask

In [104]:
projection_mask = Mask.from_cache(cache,structure_ids=167587189,hemisphere_id=3)

In [107]:
    #it is present as both a child of structure 277 in ontological order, and also as 278 in ontological order
    sid0 = list(connectivity_data.structure_datas.keys())[0]
    #eid0 = list(connectivity_data.structure_datas[sid0].experiment_datas.keys())[0]
    levs = projection_mask.reference_space.structure_tree.child_ids(
        ontological_order)
    flat_list = np.asarray([item for sublist in levs for item in sublist])

    nss = len(levs)
    leavves = np.asarray([])
    for i in range(nss):
        if np.isin(1098, levs[i]):
            print(i,levs[i],Counter(leavves)[1098])
        if len(levs[i]) > 0:
            leavves = np.append(leavves, levs[i])
        else:
            leavves = np.append(leavves, ontological_order[i])
        if np.isin(1098, levs[i]):
            print(i,levs[i],Counter(leavves)[1098])
        if np.isin(1098,ontological_order[i]):
            print('duh',i ,ontological_order[i])

277 [1098, 1107] 0
277 [1098, 1107] 1
duh 278 1098


In [101]:
len(ontological_order)

315

In [95]:
    #it is present as both a child of structure 277 in ontological order, and also as 278 in ontological order
    sid0 = list(connectivity_data.structure_datas.keys())[0]
    #eid0 = list(connectivity_data.structure_datas[sid0].experiment_datas.keys())[0]
    levs = connectivity_data.structure_datas[sid0].projection_mask.reference_space.structure_tree.child_ids(
        ontological_order)
    flat_list = np.asarray([item for sublist in levs for item in sublist])

    nss = len(levs)
    leavves = np.asarray([])
    for i in range(nss):
        if np.isin(1098, levs[i]):
            print(i,levs[i],Counter(leavves)[1098])
        if len(levs[i]) > 0:
            leavves = np.append(leavves, levs[i])
        else:
            leavves = np.append(leavves, ontological_order[i])
        if np.isin(1098, levs[i]):
            print(i,levs[i],Counter(leavves)[1098])
        if np.isin(1098,ontological_order[i]):
            print('duh',i )

277 [1098, 1107] 0
277 [1098, 1107] 1
duh 278


In [30]:
from collections import Counter

In [37]:
np.asarray(list(Counter(ontological_order_leaves).values())).argsort()[-2:]

array([525, 526])

In [38]:
ontological_order_leaves[525], ontological_order_leaves[526]

(1098.0, 1107.0)

In [99]:
ia_map[ontological_order[277]], ia_map[ontological_order[278]], ia_map[1098]

('MDRN', 'MDRNd', 'MDRNd')

In [ ]:
ontological_order = get_ordered_summary_structures(cache)


In [ ]:
#MDRNd is listed as a summary structure in the new ontology,
#but is still a child of 'MDRN', which is also listed as a summary structure

In [100]:
connectivity_data.structure_datas[sid0].projection_mask.reference_space

In [50]:


def get_leaves_ontologicalorder(connectivity_data, ontological_order):
    '''

    :param msvd:
    :param ontological_order:
    :return: The leaf order associated with the 'ontological_order' of summary structures
    '''
    sid0 = list(connectivity_data.structure_datas.keys())[0]
    #eid0 = list(connectivity_data.structure_datas[sid0].experiment_datas.keys())[0]
    levs = connectivity_data.structure_datas[sid0].projection_mask.reference_space.structure_tree.child_ids(
        ontological_order)
    flat_list = np.asarray([item for sublist in levs for item in sublist])

    nss = len(levs)
    leavves = np.asarray([])
    for i in range(nss):
        if len(levs[i]) > 0:
            leavves = np.append(leavves, levs[i])
        else:
            leavves = np.append(leavves, ontological_order[i])
    return (leavves)


277 [1098, 1107] 0
277 [1098, 1107] 1
duh


In [ ]:
ontological_order

In [87]:
connectivity_data.structure_datas[sid0].projection_mask.reference_space.structure_tree.child_ids(
        [395])

[[1098, 1107]]

In [88]:
ia_map[395]

'MDRN'

In [72]:
ontological_order[277]

395

In [80]:
ontological_order.shape, [ia_map[ontological_order[i]] for i in range(len(ontological_order))]

((315,),
 ['FRP',
  'MOp',
  'MOs',
  'SSp-n',
  'SSp-bfd',
  'SSp-ll',
  'SSp-m',
  'SSp-ul',
  'SSp-tr',
  'SSp-un',
  'SSs',
  'GU',
  'VISC',
  'AUDd',
  'AUDp',
  'AUDpo',
  'AUDv',
  'VISal',
  'VISam',
  'VISl',
  'VISp',
  'VISpl',
  'VISpm',
  'VISli',
  'VISpor',
  'ACAd',
  'ACAv',
  'PL',
  'ILA',
  'ORBl',
  'ORBm',
  'ORBvl',
  'AId',
  'AIp',
  'AIv',
  'RSPagl',
  'RSPd',
  'RSPv',
  'VISa',
  'VISrl',
  'TEa',
  'PERI',
  'ECT',
  'MOB',
  'AOB',
  'AON',
  'TT',
  'DP',
  'PIR',
  'NLOT',
  'COAa',
  'COAp',
  'PAA',
  'TR',
  'CA1',
  'CA2',
  'CA3',
  'DG',
  'FC',
  'IG',
  'ENTl',
  'ENTm',
  'PAR',
  'POST',
  'PRE',
  'SUB',
  'ProS',
  'HATA',
  'APr',
  'CLA',
  'EPd',
  'EPv',
  'LA',
  'BLA',
  'BMA',
  'PA',
  'CP',
  'ACB',
  'FS',
  'OT',
  'LSc',
  'LSr',
  'LSv',
  'SF',
  'SH',
  'AAA',
  'BA',
  'CEA',
  'IA',
  'MEA',
  'GPe',
  'GPi',
  'SI',
  'MA',
  'MS',
  'NDB',
  'TRS',
  'BST',
  'BAC',
  'VAL',
  'VM',
  'VPL',
  'VPLpc',
  'VPM',
  'VPMpc',

In [81]:
hhh = np.asarray([ia_map[ontological_order[i]] for i in range(len(ontological_order))])

In [84]:
np.where(hhh == 'stp')

(array([], dtype=int64),)

In [71]:
levs[277]

[1098, 1107]

In [70]:
connectivity_data.structure_datas[sid0].projection_mask.reference_space.structure_tree.child_ids(
        [1098])

[[]]

In [73]:
connectivity_data.structure_datas[sid0].projection_mask.reference_space.structure_tree.child_ids(
        [395])

[[1098, 1107]]

In [75]:
ia_map[395], ia_map[277], ia_map[1098]

('MDRN', 'stp', 'MDRNd')

In [76]:
ia_map[278]

'sAMY'

In [69]:
np.isin(1098,ontological_order)

array(True)

2

In [62]:
np.asarray(list(Counter(leavves).values())).argsort()[-2:]

array([525, 526])

In [54]:
Counter(oll)

Counter({68.0: 1,
         667.0: 1,
         526157192.0: 1,
         526157196.0: 1,
         526322264.0: 1,
         320.0: 1,
         943.0: 1,
         648.0: 1,
         844.0: 1,
         882.0: 1,
         656.0: 1,
         962.0: 1,
         767.0: 1,
         1021.0: 1,
         1085.0: 1,
         558.0: 1,
         838.0: 1,
         654.0: 1,
         702.0: 1,
         889.0: 1,
         929.0: 1,
         981.0: 1,
         201.0: 1,
         1047.0: 1,
         1070.0: 1,
         1038.0: 1,
         1062.0: 1,
         1030.0: 1,
         113.0: 1,
         1094.0: 1,
         1128.0: 1,
         478.0: 1,
         510.0: 1,
         878.0: 1,
         657.0: 1,
         950.0: 1,
         974.0: 1,
         1102.0: 1,
         2.0: 1,
         450.0: 1,
         854.0: 1,
         577.0: 1,
         625.0: 1,
         945.0: 1,
         1026.0: 1,
         1006.0: 1,
         670.0: 1,
         1086.0: 1,
         1111.0: 1,
         9.0: 1,
         461.0: 1,
    

In [41]:

    sid0 = list(connectivity_data.structure_datas.keys())[0]
    #eid0 = list(connectivity_data.structure_datas[sid0].experiment_datas.keys())[0]
    levs = connectivity_data.structure_datas[sid0].projection_mask.reference_space.structure_tree.child_ids(
        ontological_order)
    #flat_list = np.asarray([item for sublist in levs for item in sublist])



In [48]:
    nss = len(levs)
    leavves = np.asarray([])
    for i in range(nss):
        if np.isin(1098, levs[i]):
            print(i)

277


In [49]:
levs[277]

[1098, 1107]

In [45]:
ontological_order.shape

(315,)

In [39]:
ia_map[1098], ia_map[1107]

('MDRNd', 'MDRNv')

564

(565,)

((564,), (566,))

In [57]:
np.where(ontological_order_leaves == 1098.0)

(array([525, 527]),)

In [55]:
Counter(ontological_order_leaves)

Counter({68.0: 1,
         667.0: 1,
         526157192.0: 1,
         526157196.0: 1,
         526322264.0: 1,
         320.0: 1,
         943.0: 1,
         648.0: 1,
         844.0: 1,
         882.0: 1,
         656.0: 1,
         962.0: 1,
         767.0: 1,
         1021.0: 1,
         1085.0: 1,
         558.0: 1,
         838.0: 1,
         654.0: 1,
         702.0: 1,
         889.0: 1,
         929.0: 1,
         981.0: 1,
         201.0: 1,
         1047.0: 1,
         1070.0: 1,
         1038.0: 1,
         1062.0: 1,
         1030.0: 1,
         113.0: 1,
         1094.0: 1,
         1128.0: 1,
         478.0: 1,
         510.0: 1,
         878.0: 1,
         657.0: 1,
         950.0: 1,
         974.0: 1,
         1102.0: 1,
         2.0: 1,
         450.0: 1,
         854.0: 1,
         577.0: 1,
         625.0: 1,
         945.0: 1,
         1026.0: 1,
         1006.0: 1,
         670.0: 1,
         1086.0: 1,
         1111.0: 1,
         9.0: 1,
         461.0: 1,
    

In [56]:
np.where(ontological_order_leaves == 1098)

(array([525, 527]),)

In [20]:
ipsi_target_regions[permutation(ipsi_target_regions)]

IndexError: index 565 is out of bounds for axis 0 with size 564

In [17]:
permutation(ipsi_target_regions).shape

(566,)

In [18]:
permutation(contra_target_regions).shape

(561,)

In [19]:
contra_target_regions[permutation(contra_target_regions)]

IndexError: index 560 is out of bounds for axis 0 with size 559

In [30]:
connectivity_data.get_regionalized_normalized_data(ontological_order, ipsi_targetkey, contra_targetkey)
connectivity_data.get_creleaf_combos()
connectivity_data.leaf2_index_matrices = get_indices_2ormore(connectivity_data.leafs)
connectivity_data.creleaf2_index_matrices = get_indices_2ormore(connectivity_data.creleaf_combos)

NameError: name 'ipsi_targetkey' is not defined